<link href="https://fonts.googleapis.com/css2?family=Inter:wght@400&display=swap" rel="stylesheet">
<div style="font-family: 'Inter'; font-size: 24px; color: #749857;"><B>🍏 Show Up</b> for Health</div>
<div style="font-family: 'Inter'; font-size: 14px; color: #7a7979;"><B>Predicting Missed Apppointment in Primary Care</b> - Brompton Health PCN</div>

[janduplessis883](https://github.com/janduplessis883)<BR>
[AlexAlexRose](https://github.com/AlexAlexRose)<BR>
[FabySp](https://github.com/FabySp)<BR>
[mogleyza](https://github.com/mogleyza)

# FabySp | Hashed-ID


### Importing Libraries

In [1]:
# Importing default Libraries
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np
import seaborn as sns
import warnings

# Hi-resolution Plots and Matplotlib inline
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

# Set the maximum number of rows and columns to be displayed
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000
warnings.filterwarnings('ignore')

# "magic commands" to enable autoreload of your imported packages
%load_ext autoreload
%autoreload 2

### Params

In [2]:
RAW_DATA = '~/code/janduplessis883/data-showup/data/raw-data/'
WEATHER_DATA = '~/code/janduplessis883/data-showup/data/weather/weather.csv'
IMD_DATA = '~/code/janduplessis883/data-showup/data/imd-master/imd_master.csv'

OUTPUT_DATA = '~/code/janduplessis883/data-showup/data/output-data/'

In [3]:
# Adding the function to merge the entire dataset available with weather dataset

def make_appointments_register(surgery_list = ['ECS', 'TCP', 'TGP', 'SMW', 'KMC', 'HPVM']):
    all_surgery_list = []
    
    for surgery in surgery_list:
        surgery_appointment_list = []
        
        for i in range(1,10):
            app_path = f'{RAW_DATA}{surgery}/{surgery}_app{i}.csv'
            single_app_file = pd.read_csv(app_path)
            surgery_appointment_list.append(single_app_file)
            
        surgery_apps_df = pd.concat(surgery_appointment_list, ignore_index=True)
        #print(f'The total rows of the {surgery} surgery is {surgery_apps_df.shape[0]}')
        
        all_surgery_list.append(surgery_apps_df)
        
    full_list = pd.concat(all_surgery_list, ignore_index=True)
    return full_list
make_appointments_register()

,Appointment booked date,Appointment date,Appointment status,Appointment time,Booked by,Clinician,Rota type,Patient ID
0,10-May-22,14-Jul-22,Finished,10:00 - 10:59,"FLORKO, Aleksandra (Miss)","FLORKO, Aleksandra (Miss)",Earls Court Surgery Registrar,19581842
1,12-May-22,05-Jul-22,Finished,09:00 - 09:59,"FLORKO, Aleksandra (Miss)","FLORKO, Aleksandra (Miss)",Earls Court Surgery Nurse,19580589
2,17-May-22,05-Jul-22,Finished,10:00 - 10:59,"LE TELLIER, Jean-Pierre","FLORKO, Aleksandra (Miss)",Earls Court Surgery Nurse,47551135
3,31-May-22,07-Jul-22,Finished,11:00 - 11:59,"FRANKSON, Joyce (Ms)","ESAYU, Lula",Earls Court Surgery HCA,19579593
4,07-Jun-22,05-Jul-22,Did Not Attend,11:00 - 11:59,"FLORKO, Aleksandra (Miss)","FLORKO, Aleksandra (Miss)",Earls Court Surgery Nurse,60020944
...,...,...,...,...,...,...,...,...
877462,31-Dec-19,31-Dec-19,Finished,15:00 - 15:59,"AMADI, Kasi (Miss)","PATEL, Rinku (Pharmacist)",Redcliffe Pharmacist,47490926
877463,31-Dec-19,31-Dec-19,Finished,15:00 - 15:59,"AMADI, Kasi (Miss)","PATEL, Rinku (Pharmacist)",Redcliffe Pharmacist,47493202
877464,31-Dec-19,31-Dec-19,Finished,15:00 - 15:59,"AMADI, Kasi (Miss)","PATEL, Rinku (Pharmacist)",Redcliffe Pharmacist,47487575
877465,31-Dec-19,31-Dec-19,Finished,15:00 - 15:59,"DUFFIELD, Katie (Miss)","PATEL, Rinku (Pharmacist)",Redcliffe Pharmacist,47489391


In [4]:
df = make_appointments_register()

In [5]:
df.head()

,Appointment booked date,Appointment date,Appointment status,Appointment time,Booked by,Clinician,Rota type,Patient ID
0,10-May-22,14-Jul-22,Finished,10:00 - 10:59,"FLORKO, Aleksandra (Miss)","FLORKO, Aleksandra (Miss)",Earls Court Surgery Registrar,19581842
1,12-May-22,05-Jul-22,Finished,09:00 - 09:59,"FLORKO, Aleksandra (Miss)","FLORKO, Aleksandra (Miss)",Earls Court Surgery Nurse,19580589
2,17-May-22,05-Jul-22,Finished,10:00 - 10:59,"LE TELLIER, Jean-Pierre","FLORKO, Aleksandra (Miss)",Earls Court Surgery Nurse,47551135
3,31-May-22,07-Jul-22,Finished,11:00 - 11:59,"FRANKSON, Joyce (Ms)","ESAYU, Lula",Earls Court Surgery HCA,19579593
4,07-Jun-22,05-Jul-22,Did Not Attend,11:00 - 11:59,"FLORKO, Aleksandra (Miss)","FLORKO, Aleksandra (Miss)",Earls Court Surgery Nurse,60020944


In [8]:
import hashlib

def hash_patient_id(df, length=8):
    df['Patient ID'] = df['Patient ID'].apply(lambda x: hashlib.sha512(str(x).encode('utf-8')).hexdigest()[:length])
    return df

hash_patient_id(df)


,Appointment booked date,Appointment date,Appointment status,Appointment time,Booked by,Clinician,Rota type,Patient ID
0,10-May-22,14-Jul-22,Finished,10:00 - 10:59,"FLORKO, Aleksandra (Miss)","FLORKO, Aleksandra (Miss)",Earls Court Surgery Registrar,4054bd9d
1,12-May-22,05-Jul-22,Finished,09:00 - 09:59,"FLORKO, Aleksandra (Miss)","FLORKO, Aleksandra (Miss)",Earls Court Surgery Nurse,bd36328c
2,17-May-22,05-Jul-22,Finished,10:00 - 10:59,"LE TELLIER, Jean-Pierre","FLORKO, Aleksandra (Miss)",Earls Court Surgery Nurse,4f60d3ba
3,31-May-22,07-Jul-22,Finished,11:00 - 11:59,"FRANKSON, Joyce (Ms)","ESAYU, Lula",Earls Court Surgery HCA,5140764e
4,07-Jun-22,05-Jul-22,Did Not Attend,11:00 - 11:59,"FLORKO, Aleksandra (Miss)","FLORKO, Aleksandra (Miss)",Earls Court Surgery Nurse,2e7c7732
...,...,...,...,...,...,...,...,...
877462,31-Dec-19,31-Dec-19,Finished,15:00 - 15:59,"AMADI, Kasi (Miss)","PATEL, Rinku (Pharmacist)",Redcliffe Pharmacist,be93487e
877463,31-Dec-19,31-Dec-19,Finished,15:00 - 15:59,"AMADI, Kasi (Miss)","PATEL, Rinku (Pharmacist)",Redcliffe Pharmacist,c72b6938
877464,31-Dec-19,31-Dec-19,Finished,15:00 - 15:59,"AMADI, Kasi (Miss)","PATEL, Rinku (Pharmacist)",Redcliffe Pharmacist,6ef8fa8b
877465,31-Dec-19,31-Dec-19,Finished,15:00 - 15:59,"DUFFIELD, Katie (Miss)","PATEL, Rinku (Pharmacist)",Redcliffe Pharmacist,cf1805d6


In [ ]:
# import hashlib

# # Function to hash values using SHA-256 and truncate the result
# def hash_and_truncate(value, length=8):
#     # Convert the value to a string
#     value_str = str(value)

#     # Create a hash object using SHA-256
#     sha256 = hashlib.sha256()

#     # Update the hash object with the value
#     sha256.update(value_str.encode('utf-8'))

#     # Get the hexadecimal representation of the hash and truncate it
#     hashed_value = sha256.hexdigest()[:length]
    
#     # Apply the hash function to the 'Patient_ID' column and create a new 'Hashed_Patient_ID' column
#     df['Hashed Patient ID'] = df['Patient ID'].apply(hash_and_truncate)

#     # Drop the original 'Patient_ID' column if you no longer need it
#     df.drop(columns=['Patient ID'], inplace=True)
    
#     return df


# hash_and_truncate(df)

In [ ]:
import math

def haversine_distance(surgery_prefix, lat2, lon2):
    R = 6371.0  # Radius of the Earth in kilometers

    if surgery_prefix == 'ECS':
        lat1, lon1 = 51.488721, -0.191873
    elif surgery_prefix == 'SMW':
        lat1, lon1 = 51.494474, -0.181931
    elif surgery_prefix == 'TCP':
        lat1, lon1 = 51.48459, -0.171887
    elif surgery_prefix == 'HPVM':
        lat1, lon1 = 51.48459, -0.171887
    elif surgery_prefix == 'KMC':
        lat1, lon1 = 51.49807, -0.159918
    elif surgery_prefix == 'TGP':
        lat1, lon1 = 51.482652, -0.178066


    # Convert degrees to radians
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    # Differences
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    # Haversine formula
    a = math.sin(dlat / 2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = R * c
    return distance  # in kilometers

In [ ]:
def make_global_disease_register(surgery_list = ['ECS', 'TCP', 'TGP', 'SMW', 'KMC', 'HPVM']):
    print('=== Preparing Global Disease Register + IMD2023 info ======================================')

    disease_register = []
    for surgery in surgery_list:
        register_path = f'{RAW_DATA}{surgery}/{surgery}'

        idnhs = pd.read_excel(f'{register_path}_NHS_PTID.xlsx', dtype='str')
        idnhs.dropna(inplace=True)
        frail = pd.read_csv(f'{register_path}_FRAILTY.csv', dtype='str')
        dep = pd.read_csv(f'{register_path}_DEPRESSION.csv', dtype='str')
        obesity = pd.read_csv(f'{register_path}_OBESITY.csv', dtype='str')
        chd = pd.read_csv(f'{register_path}_IHD.csv', dtype='str')
        dm = pd.read_csv(f'{register_path}_DM.csv', dtype='str')
        hpt = pd.read_csv(f'{register_path}_HPT.csv', dtype='str')
        ndhg = pd.read_csv(f'{register_path}_NDHG.csv', dtype='str')
        smi = pd.read_csv(f'{register_path}_SMI.csv', dtype='str')

        ptid = idnhs.merge(frail, how='left', on='NHS number')
        ptid = ptid.drop(columns='NHS number')

        register = (ptid.merge(dep, how='left', on='Patient ID')
                    .merge(obesity, how='left', on='Patient ID')
                    .merge(chd, how='left', on='Patient ID')
                    .merge(dm, how='left', on='Patient ID')
                    .merge(hpt, how='left', on='Patient ID')
                    .merge(ndhg, how='left', on='Patient ID')
                    .merge(smi, how='left', on='Patient ID')
                    .fillna(0)
                    )
        print(f'💊 {surgery} Disease Register completed')
        # Add IMD and distance from station
        imd = pd.read_csv(IMD_DATA)

        full_register = register.merge(imd, how='left', on='Postcode')
        print(f'🔸 {surgery} IMD2023')
        full_register['distance_from_surg'] = full_register.apply(lambda row: haversine_distance(surgery, row['Latitude'], row['Longitude']), axis=1)
        disease_register.append(full_register)

    global_register = pd.concat(disease_register, axis=0, ignore_index=True)
    print(f"🦠 Concat Registers into ONE REGISTER")
    
    global_register.dropna(inplace=True)
    print(f'❌ Dropped NaN')
    
    output_path = f'{OUTPUT_DATA}global_disease_register.csv'
    global_register.to_csv(output_path, index=False)
    print(f'✅ Global Disease Register Saved to output-data: {global_register.shape}')
    print()
    return global_register

In [ ]:
disease_register = make_global_disease_register()